# Урок 9. Интеграция. Итоговый проект

# Домашнее задание

**Стандартная версия**
Нужно реализовать rest api на базе flask в google colab.

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. Реализовать ноутбук с сервером
3. Реализовать ноутбук с клиентом


**Сложная версия**

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта)
3. итоговый проект должен содержать:
    1. каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна)
    2. файл app/run_server.py (здесь основной код flask-приложения)
    3. requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта)
    4. README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д)
    5. *Dockerfile
    6. *docker-entrypoint.sh
4. *front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction
2. Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [1]:
!pip install dill

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import FeatureUnion

Загрузим данные

In [2]:
df = pd.read_csv('C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lection3/train_case2.csv', ';')
df.head(3)

C:\Users\vkhur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
df['cardio'].value_counts()

0    35021
1    34979
Name: cardio, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df['cardio'],
                                                    test_size=0.33, random_state=42)
# save test
X_test.to_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/X_test.csv", index=None)
y_test.to_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/y_test.csv", index=None)

# save train
X_train.to_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/X_train.csv", index=None)
y_train.to_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/y_train.csv", index=None)

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:
features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']
target = 'cardio'

Соберем кусок, ответственный за feature engineering

Добавим простейший классификатор

In [8]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 1.01749796, -1.01586151, -0.986831  , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.3334502 , -0.65065191, -0.986831  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.38653685,  0.68844994,  1.10127359, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67439454,  0.56671341, -0.01238219, ...,  0.        ,
         0.        ,  1.        ],
       [-0.49851073,  0.32324034, -0.29079614, ...,  0.        ,
         0.        ,  0.        ],
       [-1.76002771,  1.54060566, -0.70841706, ...,  0.        ,
         0.        ,  1.        ]])

In [9]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: total: 203 ms
Wall time: 210 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                        

Сохраним модель (пайплайн)

In [10]:
with open("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

# Step 2 - PREDICT

### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [11]:
X_test = pd.read_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/X_test.csv")
y_test = pd.read_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/y_test.csv")

In [12]:
X_test.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1,1


In [13]:
with open('C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [14]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('height',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='height')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('weight',
                                                 Pipeline(steps=[('selector',
                                        

In [15]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/test_predictions.csv", index=None)

In [16]:
preds[:10]

array([0.73039568, 0.81042059, 0.69781561, 0.50673558, 0.34064977,
       0.42416665, 0.24444879, 0.47473676, 0.93293358, 0.76533498])

In [17]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.39736024537833337, F-Score=0.735, Precision=0.653, Recall=0.840


# Step 3 - FLASK

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование.

## Flask

Тут будет сервис для обработки запросов на Flask

Google Colab предоставляет виртуальную машину, поэтому мы не можем получить доступ к локальному хосту, как это делаем на нашем локальном компьютере при запуске локального веб-сервера. Что мы можем сделать, так это предоставить его общедоступному URL-адресу с помощью ngrok.

https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [20]:
!pip3 install flask-ngrok

In [18]:
#from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [ ]:
# Пробный запуск Flask

app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run(port=8082)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8082/ (Press CTRL+C to quit)


In [19]:
import pandas as pd
import dill

### **Создаем сервис для обработки запросов к модели**

In [20]:
# Загружаем обученные модели
with open('C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [21]:
X_test = pd.read_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/X_test.csv")
y_test = pd.read_csv("C:/Users/vkhur/Desktop/Учеба/Машинное обучение в бизнесе/Lessonn9/y_test.csv")

Запустить сервис и не глушить его, пока работаем 

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke, alco, active  = "", "", "", "", "", "", "", "", "", "", ""
    
    request_json = request.get_json()
    
    if request_json["age"]:
        age = request_json['age']
    
    if request_json["gender"]:
        gender = request_json['gender']
                
    if request_json["height"]:
        height = request_json['height']
        
    if request_json["weight"]:
        weight = request_json['weight']
    
    if request_json["ap_hi"]:
        ap_hi = request_json['ap_hi']
                
    if request_json["ap_lo"]:
        ap_lo = request_json['ap_lo']
        
    if request_json["cholesterol"]:
        cholesterol = request_json['cholesterol']
                
    if request_json["gluc"]:
        gluc = request_json['gluc']
        
    if request_json["smoke"]:
        smoke = request_json['smoke']
    
    if request_json["alco"]:
        alco = request_json['alco']
                
    if request_json["active"]:
        active = request_json['active']
    
    print(description)  
    preds = model.predict_proba(pd.DataFrame({'age': [age],
                                              'gender': [gender],
                                              'height': [height],
                                              'weight': [weight], 
                                              'ap_hi': [ap_hi],
                                              'ap_lo': [ap_lo], 
                                              'cholesterol': [cholesterol], 
                                              'gluc': [gluc],
                                              'smoke': [smoke],
                                              'alco': [alco],
                                              'active': [active]
                                             }))
    data["predictions"] = preds[:, 1][0]
    data["gender"] = gender
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run(port=8082)

In [ ]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data